In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

url="https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"

page=requests.get(url)

soup=BeautifulSoup(page.text, "html")
#print(soup)

In [2]:
#another table already exists before the table we want
table=soup.find_all("table")[1]
#get 4th elements onwards
specified_titles=table.find_all("th")[4:]
cleaned_specified_titles=[title.text.strip() for title in specified_titles][0:16] #keep only the first 15 elements
print(cleaned_specified_titles)
#change names at given indeces
cleaned_specified_titles[2::5]=["Summer_Gold","Winter_Gold","Combined_Gold"]
cleaned_specified_titles[3::5]=["Summer_Silver","Winter_Silver","Combined_Silver"]
cleaned_specified_titles[4::5]=["Summer_Bronze","Winter_Bronze","Combined_Bronze"]
cleaned_specified_titles[5::5]=["Summer_Total","Winter_Total","Combined_Total"]
#print(cleaned_specified_titles)


['Team (IOC code)', 'No.', '', '', '', '', 'No.', '', '', '', '', 'No.', '', '', '', '']


In [3]:
#each element in the list is 1 row from each table
data_draft=table.find_all("tr")[2:]#first 2 rows are column names which arent needed
data_draft=data_draft[:-1]#the last row is a cumulative total, only has "th" tags and will not have "td" tags so 
            #last element of the following list will be empty. Thus we keep all elements except last one

#empty list to hold values
frame_rows=[]

for row in data_draft:
    #find all "td" tags
    row_data=row.find_all("td")
    #clean up every element using list comprehension
    row_data_clean=[data.text.strip() for data in row_data]
    
    #remove any chars after the ")" using split and add back the ")"
    #EX:"Australiia (AUS)[AUS][Z]" gets split into ["Australiia (AUS" , "[AUS][Z]"]. We only need element 
    #at index 0, then add a ")" at end. "Australia (AUS" + ")" = "Australia (AUS)"
    fixed=row_data_clean[0].split(")")[0]+")"
    
    #add clean element into the first index of the list
    row_data_clean[0]=fixed
    
    frame_rows.append(row_data_clean)

#print(frame_rows)

In [4]:
df=pd.DataFrame(frame_rows, columns=cleaned_specified_titles)

In [5]:
df

,Team (IOC code),No.,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,No.,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,No.,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total
0,Afghanistan (AFG),16,0,0,2,2,0,0,0,0,0,16,0,0,2,2
1,Albania (ALB),10,0,0,2,2,5,0,0,0,0,15,0,0,2,2
2,Algeria (ALG),15,7,4,9,20,3,0,0,0,0,18,7,4,9,20
3,Argentina (ARG),26,22,27,31,80,20,0,0,0,0,46,22,27,31,80
4,Armenia (ARM),8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,Zimbabwe (ZIM),15,3,4,1,8,1,0,0,0,0,16,3,4,1,8
158,Individual Neutral Athletes (AIN),1,1,3,1,5,0,0,0,0,0,1,1,3,1,5
159,Independent Olympic Athletes (IOA),3,1,0,1,2,0,0,0,0,0,3,1,0,1,2
160,Independent Olympic Participants (IOP),1,0,1,2,3,0,0,0,0,0,1,0,1,2,3


In [6]:
#rename column
df.rename(columns={"Team (IOC code)":"Country"},inplace=True)

#FOR BELOW I USED: https://stackoverflow.com/questions/41685917/rename-dataframe-column-based-on-column-index
#If using .rename; esentially since the columns' names are duplicated, it will rename all the duplicates. 
#Even if we specify column index. To avoid that we can do this:
df.columns.values[1] = "Summer_Olympics_Attended"
df.columns.values[6]= "Winter_Olympics_Attended"
df.columns.values[11]= "Total_Olympics_Attended"
df

,Country,Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,Total_Olympics_Attended,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total
0,Afghanistan (AFG),16,0,0,2,2,0,0,0,0,0,16,0,0,2,2
1,Albania (ALB),10,0,0,2,2,5,0,0,0,0,15,0,0,2,2
2,Algeria (ALG),15,7,4,9,20,3,0,0,0,0,18,7,4,9,20
3,Argentina (ARG),26,22,27,31,80,20,0,0,0,0,46,22,27,31,80
4,Armenia (ARM),8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,Zimbabwe (ZIM),15,3,4,1,8,1,0,0,0,0,16,3,4,1,8
158,Individual Neutral Athletes (AIN),1,1,3,1,5,0,0,0,0,0,1,1,3,1,5
159,Independent Olympic Athletes (IOA),3,1,0,1,2,0,0,0,0,0,3,1,0,1,2
160,Independent Olympic Participants (IOP),1,0,1,2,3,0,0,0,0,0,1,0,1,2,3


In [7]:
#save this original df as a csv(NOTE: For some reason, there is an A with a carrot on top after the country name when opening in excel. 
#Notepad doesnt have this issue)
#df.to_csv(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Original.csv", index=False)

#save this original df as an excel file
#df.to_excel(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Original.xlsx", index=False)

In [8]:
df.dtypes

Country                     object
Summer_Olympics_Attended    object
Summer_Gold                 object
Summer_Silver               object
Summer_Bronze               object
Summer_Total                object
Winter_Olympics_Attended    object
Winter_Gold                 object
Winter_Silver               object
Winter_Bronze               object
Winter_Total                object
Total_Olympics_Attended     object
Combined_Gold               object
Combined_Silver             object
Combined_Bronze             object
Combined_Total              object
dtype: object

In [9]:
df.columns

#FOR DATA TYPE CHANGES:
#https://stackoverflow.com/questions/15891038/change-column-type-in-pandas

#METHOD 1: CHANGE DATA TYPES()

#used convert_dtypes to allow pandas to infer what the right dtype is. it chose string first
newdf=df.convert_dtypes()

#If just changing one column, could just use the line below instead of looping
#newdf["Summer_Gold"]=newdf["Summer_Gold"].str.replace(',', '')


#get all the cols except the col at index 0 which is the country column
newdf_num_cols=newdf.columns[1:]

#the ',' in some values will not allow all cols to be changed to int data type, so we have to get rid of the ','
for col in newdf_num_cols:
    newdf[col]=newdf[col].str.replace(",","")

#change the types from string to int for the chosen cols
newdf[newdf_num_cols] = newdf[newdf_num_cols].apply(pd.to_numeric)


newdf.dtypes

Country                     string[python]
Summer_Olympics_Attended             Int64
Summer_Gold                          Int64
Summer_Silver                        Int64
Summer_Bronze                        Int64
Summer_Total                         Int64
Winter_Olympics_Attended             Int64
Winter_Gold                          Int64
Winter_Silver                        Int64
Winter_Bronze                        Int64
Winter_Total                         Int64
Total_Olympics_Attended              Int64
Combined_Gold                        Int64
Combined_Silver                      Int64
Combined_Bronze                      Int64
Combined_Total                       Int64
dtype: object

In [10]:
#sort the countries in descending order based on total gold medals
success=newdf.sort_values(by=["Combined_Gold"], ascending=False)
success

,Country,Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,Total_Olympics_Attended,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total
149,United States (USA),29,1105,879,780,2764,24,114,121,95,330,53,1219,1000,875,3094
114,Soviet Union (URS),9,395,319,296,1010,9,78,57,59,194,18,473,376,355,1204
26,China (CHN),12,303,226,198,727,12,22,32,23,77,24,325,258,221,804
49,Germany (GER),18,213,220,255,688,13,104,98,65,267,31,317,318,320,955
54,Great Britain (GBR),30,298,340,343,981,24,12,5,17,34,54,310,345,360,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Paraguay (PAR),14,0,1,0,1,1,0,0,0,0,15,0,1,0,1
110,Refugee Olympic Team (EOR),3,0,0,1,1,0,0,0,0,0,3,0,0,1,1
118,Samoa (SAM),11,0,1,0,1,0,0,0,0,0,11,0,1,0,1
120,San Marino (SMR),16,0,1,2,3,11,0,0,0,0,27,0,1,2,3


## Add "Continent" column and populate values

In [11]:
# Webscrape to get lists of countries in each continent
#create lists of countries in each continent
continents_url="https://simple.wikipedia.org/wiki/List_of_countries_by_continents"

page_continents=requests.get(continents_url)

continent_soup=BeautifulSoup(page_continents.text, "html")
#get a list of all the tables in webpage
Tables_list=continent_soup.find_all("table")

#GETTING AFRICAN COUNTRIES

#Part 1(sovereign countries)

#get a list of all "td" tags in the table at index 0 and then keep the values starting at index 2 and then have a step
#of 8 (EX. get index 2, then index 10..until the end)
africa_countries1=Tables_list[0].find_all("td")[2::8]

#clean up list by removing tags
cleaned_africa_countries1=[country.text.strip() for country in africa_countries1]

#go through each item in the list and remove any footnotes(some countries have a "[]" next to it )
for i in range(len(cleaned_africa_countries1)):
    cleaned_africa_countries1[i]=cleaned_africa_countries1[i].split("[")[0]

#Part 2(dependent territories)
#get a list of all "td" tags in the table at index 0 and then keep the values starting at index 1 and then have a step
#of 7 (EX. get index 1, then index 8..until the end)
africa_countries2=Tables_list[1].find_all("td")[1::7]

#clean up list by removing tags
cleaned_africa_countries2=[country.text.strip() for country in africa_countries2]

#join both
cleaned_africa_countries=cleaned_africa_countries1+cleaned_africa_countries2


#GETTING ASIAN COUNTRIES
asia_countries=Tables_list[2].find_all("td")[2::7]
cleaned_asia_countries=[country.text.strip() for country in asia_countries]

for i in range(len(cleaned_asia_countries)):
    cleaned_asia_countries[i]=cleaned_asia_countries[i].split("[")[0]

    
#GETTING EUROPEAN COUNTRIES
#some 'td' values were extras so we had to use starting from 3rd index
europe_countries=Tables_list[3].find_all("td")[3::8]
cleaned_europe_countries=[country.text.strip() for country in europe_countries]

for i in range(len(cleaned_europe_countries)):
    cleaned_europe_countries[i]=cleaned_europe_countries[i].split("[")[0]
    

    
#GETTING NORTH AMERICAN COUNTRIES
north_countries=Tables_list[4].find_all("td")[2::8]
cleaned_north_countries=[country.text.strip() for country in north_countries]

for i in range(len(cleaned_north_countries)):
    cleaned_north_countries[i]=cleaned_north_countries[i].split("[")[0]

    

#GETTING SOUTH AMERICAN COUNTRIES
south_countries=Tables_list[5].find_all("td")[2::9]
cleaned_south_countries=[country.text.strip() for country in south_countries]
#NOTE: No for loop needed to remove footnotes, since there are no footnotes    

#GETTING OCEANIAN COUNTRIES
oceania_countries=Tables_list[6].find_all("td")[2::9]
cleaned_oceania_countries=[country.text.strip() for country in oceania_countries]
#NOTE: No for loop needed to remove footnotes, since there are no footnotes
  

In [12]:
#Fill in column using above lists
"""
#CANT DO THIS BLOCK OF CODE BECAUSE WE COULDNT MAKE ELSE DO NOTHING, AND DUE TO THAT VALUES WILL BE OVERWRITTEN
#SOURCE:https://stackoverflow.com/questions/66651910/how-to-do-nothing-with-if-else-in-a-apply-lambda-function

#for africa
newdf["Continent"] = newdf["Country"].apply(lambda x: "Africa" if any(caf in x for caf in cleaned_africa_countries) else pass)

#for asia
newdf["Continent"] = newdf["Country"].apply(lambda x: "Asia" if any(afc in x for afc in cleaned_asia_countries) else np.nan)

#for europe
newdf["Continent"] = newdf["Country"].apply(lambda x: "Europe" if any(uefa in x for uefa in cleaned_europe_countries) else np.nan)

#for north america
newdf["Continent"] = newdf["Country"].apply(lambda x: "North America" if any(concacaf in x for concacaf in cleaned_north_countries) else np.nan)

#for south america
newdf["Continent"] = newdf["Country"].apply(lambda x: "South America" if any(conmebol in x for conmebol in cleaned_south_countries) else np.nan)

#for oceania
newdf["Continent"] = newdf["Country"].apply(lambda x: "Oceania" if any(ofc in x for ofc in cleaned_oceania_countries) else np.nan)
newdf
"""


#METHOD 1: CREATE "CONTINENT" COL AND SET ALL TO NaN. THEN LOOP VARIOUS TIMES TO CHANGE VALUES BASED ON CONTINENT
newdf["Continent"] = np.nan

for caf in cleaned_africa_countries:
    newdf.loc[newdf["Country"].str.contains(caf), "Continent"]="Africa"
    
for afc in cleaned_asia_countries:
    newdf.loc[newdf["Country"].str.contains(afc), "Continent"]="Asia"

for uefa in cleaned_europe_countries:
    newdf.loc[newdf["Country"].str.contains(uefa), "Continent"]="Europe"

for concacaf in cleaned_north_countries:
    newdf.loc[newdf["Country"].str.contains(concacaf), "Continent"]="North America"

for conmebol in cleaned_south_countries:
    newdf.loc[newdf["Country"].str.contains(conmebol), "Continent"]="South America"

for ofc in cleaned_oceania_countries:
    newdf.loc[newdf["Country"].str.contains(ofc), "Continent"]="Oceania"

#Find rows where there is a NaN value in the continent column
#https://www.statology.org/pandas-select-rows-with-nan/

#some countries do not exist anymore, are written differently, shortened, are territories, or just olympic groups 
newdf.loc[newdf["Continent"].isnull()]

,Country,Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,Total_Olympics_Attended,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total,Continent
5,Australasia (ANZ),2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,NaN
9,Bahamas (BAH),18,8,2,6,16,0,0,0,0,0,18,8,2,6,16,NaN
14,Bermuda (BER),20,1,0,1,2,8,0,0,0,0,28,1,0,1,2,NaN
15,Bohemia (BOH),3,0,1,3,4,0,0,0,0,0,3,0,1,3,4,NaN
18,British West Indies (BWI),1,0,0,2,2,0,0,0,0,0,1,0,0,2,2,NaN
29,Ivory Coast (CIV),15,1,1,3,5,0,0,0,0,0,15,1,1,3,5,NaN
33,Czech Republic (CZE),8,22,22,28,72,8,10,11,13,34,16,32,33,41,106,NaN
34,Czechoslovakia (TCH),16,49,49,45,143,16,2,8,15,25,32,51,57,60,168,NaN
54,Great Britain (GBR),30,298,340,343,981,24,12,5,17,34,54,310,345,360,1015,NaN
60,Hong Kong (HKG),18,4,3,6,13,6,0,0,0,0,24,4,3,6,13,NaN


In [13]:
extradf=newdf.drop(columns="Continent")
extradf2=newdf.copy()

#METHODS 2 and 3 are probably cleaner than METHOD 1


#METHOD 2 Using dictionary
cont_country={
    "Africa":cleaned_africa_countries,
    "Asia":cleaned_asia_countries,
    "Europe":cleaned_europe_countries,
    "North America":cleaned_north_countries,
    "South America":cleaned_south_countries,
    "Oceania": cleaned_oceania_countries
}

#loop through the dictionary
for continent,country_list in cont_country.items():
    #go through each country in the country list
    for cntry in country_list:
        #check to see any countries in df that match the current country in the country list and set the "continent" as
        #the current key being looped through
        extradf.loc[extradf["Country"].str.contains(cntry), "Continent"]=continent

#METHOD 3 use 2 lists
continents=["Africa", "Asia","Europe", "North America", "South America", "Oceania"]
countries_grouped=[
    cleaned_africa_countries,
    cleaned_asia_countries,
    cleaned_europe_countries,
    cleaned_north_countries,
    cleaned_south_countries,
    cleaned_oceania_countries
]

#both lists should have same length(list 1: 6 continents, list2: 6 lists of countries)
num_groups=len(continents)
#loop through each index of each original list
for i in range(num_groups):
    #loop through the current list at the current index in "countries_grouped" list
    for member in countries_grouped[i]:
        extradf2.loc[extradf2["Country"].str.contains(member), "Continent"] = continents[i]

extradf2.loc[extradf2["Continent"].isnull()] #the rows with NaN in continent


,Country,Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,Total_Olympics_Attended,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total,Continent
5,Australasia (ANZ),2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,NaN
9,Bahamas (BAH),18,8,2,6,16,0,0,0,0,0,18,8,2,6,16,NaN
14,Bermuda (BER),20,1,0,1,2,8,0,0,0,0,28,1,0,1,2,NaN
15,Bohemia (BOH),3,0,1,3,4,0,0,0,0,0,3,0,1,3,4,NaN
18,British West Indies (BWI),1,0,0,2,2,0,0,0,0,0,1,0,0,2,2,NaN
29,Ivory Coast (CIV),15,1,1,3,5,0,0,0,0,0,15,1,1,3,5,NaN
33,Czech Republic (CZE),8,22,22,28,72,8,10,11,13,34,16,32,33,41,106,NaN
34,Czechoslovakia (TCH),16,49,49,45,143,16,2,8,15,25,32,51,57,60,168,NaN
54,Great Britain (GBR),30,298,340,343,981,24,12,5,17,34,54,310,345,360,1015,NaN
60,Hong Kong (HKG),18,4,3,6,13,6,0,0,0,0,24,4,3,6,13,NaN


In [14]:
#CLEANING UP NaN VALUES IN CONTINENT COLUMN
#EDIT values for countries/groups that have NaN value in Continent column

#Testing and only changing Continent value 
extradf2.loc[extradf2["Country"].str.contains("Australasia"), "Continent"]="Oceania"
extradf2.loc[extradf2["Country"].str.contains("Australasia")]


# NOTE: ROC(ROC)=Russian Olympic Commitee

msng_afr=["Ivory Coast"]
msng_asi=["Hong Kong", "(TPE)"]
msng_eur=["Bohemia","Czech Republic", "Czechoslovakia","Great Britain","Soviet Union", "(ROC)", "(YUG)"]
msng_na=["Bahamas", "Bermuda", "British West Indies","(PUR)", "(ISV)"]
msng_sa=[]
msng_oce=[]

msng_grouped=[
    msng_afr,
    msng_asi,
    msng_eur,
    msng_na,
    msng_sa,
    msng_oce
]

#using continents list and num_groups variable from prev block of code
for i in range(num_groups):
    for msng in msng_grouped[i]:
        
        #had to use regex=False in str.contains because the code would run but give a warning message:
        #UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
        #due to using the 3 letter country code in the missing lists [ex(YUG)]
        extradf2.loc[extradf2["Country"].str.contains(msng, regex=False), "Continent"]=continents[i]

extradf2.loc[extradf2["Continent"].isnull()] #the rows with NaN in continent

#Checking to see if value was actually changed
#extradf2.loc[extradf2["Country"].str.contains("(ROC)", regex=False)]


#OPTIONAL: CHANGE NaN values to "None"
#extradf.loc[extradf["Continent"].isnull(),"Continent"]="None"
#extradf

,Country,Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,Total_Olympics_Attended,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total,Continent
110,Refugee Olympic Team (EOR),3,0,0,1,1,0,0,0,0,0,3,0,0,1,1,NaN
115,Unified Team (EUN),1,45,38,29,112,1,9,6,8,23,2,54,44,37,135,NaN
158,Individual Neutral Athletes (AIN),1,1,3,1,5,0,0,0,0,0,1,1,3,1,5,NaN
159,Independent Olympic Athletes (IOA),3,1,0,1,2,0,0,0,0,0,3,1,0,1,2,NaN
160,Independent Olympic Participants (IOP),1,0,1,2,3,0,0,0,0,0,1,0,1,2,3,NaN
161,Mixed team (ZZX),3,11,6,8,25,0,0,0,0,0,3,11,6,8,25,NaN


In [15]:
#THIS IS FOR TESTING PURPOSES ONLY
#CREATE A NEW DF THAT CONSISTS OF ROWS THAT HAVE A VALUE IN "CONTINENT" AND ARE NOT NULL
#source: 
#https://stackoverflow.com/questions/22551403/python-pandas-filtering-out-nan-from-a-data-selection-of-a-column-of-strings
trial1=extradf2.loc[extradf2["Continent"].notnull()]


print(trial1.shape)

#getting row index for country (ROC)
roc=trial1.index[trial1["Country"].str.contains("ROC")].tolist()
roc

(156, 17)


[117]

In [16]:
#getting dimensions of extradf2
rows,cols=extradf2.shape

#Creating a copy of extradf2
w_continent=extradf2.copy()
w_continent.rename(columns={"Country":"Country (IOC)"},inplace=True)



#create 2 new columns splitting country and ioc code
#ONE SOURCE:https://stackoverflow.com/questions/14745022/how-to-split-a-dataframe-string-column-into-two-columns

#IMAGINARY SCENARIO: SUPPOSE WE ONLY WANTED THE 1ST PART OF THE SPLIT, WE COULD USE:
#w_continent["Country"]=w_continent["Country (IOC)"].str.split("(").str[0]


#METHOD 1 split into two cols and get ioc codes by replacing parentheses

#use str.split to split on "(" and then replace the  remaining ")" with ""
w_continent[["Country", "IOC"]]=w_continent["Country (IOC)"].str.split("(", expand=True)
w_continent["IOC"]=w_continent["IOC"].str.replace(")", "")
#remove leading and trailing whitespaces
w_continent["Country"]=w_continent["Country"].str.strip()



#METHOD 2 split into two extra cols and get ioc codes in parantheses through extraction 

#split on the first "(" and save to variable divid(SHOULD USE EXPAND=TRUE TO AVOID errors)
#divid = w_continent["Country (IOC)"].str.split('(', n=1, expand=True, regex=False)

#Add a parentheses to series at index 1 [The IOC column] EX[ "ARG)" becomes "(ARG)" ] which will help with the 'extract'
#method used after the following line of code
#divid[1]="("+divid[1]

#Add series values to new columns
#w_continent[["Country", "IOC"]]=divid

"""
#FOLLOWING LINE: deletes parentheses and anything inside(NOT WHAT WE WANT)
#w_continent['IOC']=w_continent['IOC'].str.replace(r"\(.*\)","",regex=True)
"""

#keep what is in the parantheses and remove parentheses from IOC col[Ex "(ARG)" becomes "ARG"]
#w_continent['IOC']=w_continent['IOC'].str.extract(r'(\w+)')


w_continent

,Country (IOC),Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,Winter_Total,Total_Olympics_Attended,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total,Continent,Country,IOC
0,Afghanistan (AFG),16,0,0,2,2,0,0,0,0,0,16,0,0,2,2,Asia,Afghanistan,AFG
1,Albania (ALB),10,0,0,2,2,5,0,0,0,0,15,0,0,2,2,Europe,Albania,ALB
2,Algeria (ALG),15,7,4,9,20,3,0,0,0,0,18,7,4,9,20,Africa,Algeria,ALG
3,Argentina (ARG),26,22,27,31,80,20,0,0,0,0,46,22,27,31,80,South America,Argentina,ARG
4,Armenia (ARM),8,2,11,9,22,8,0,0,0,0,16,2,11,9,22,Asia,Armenia,ARM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,Zimbabwe (ZIM),15,3,4,1,8,1,0,0,0,0,16,3,4,1,8,Africa,Zimbabwe,ZIM
158,Individual Neutral Athletes (AIN),1,1,3,1,5,0,0,0,0,0,1,1,3,1,5,NaN,Individual Neutral Athletes,AIN
159,Independent Olympic Athletes (IOA),3,1,0,1,2,0,0,0,0,0,3,1,0,1,2,NaN,Independent Olympic Athletes,IOA
160,Independent Olympic Participants (IOP),1,0,1,2,3,0,0,0,0,0,1,0,1,2,3,NaN,Independent Olympic Participants,IOP


In [17]:
w_continent.sort_values(by="Combined_Gold", ascending=False)

w_continent.groupby(by="Continent")[["Total_Olympics_Attended"]].mean().sort_values(by="Total_Olympics_Attended", ascending=False)

,Total_Olympics_Attended
Continent,
Europe,28.166667
South America,25.727273
Oceania,22.5
North America,22.2
Asia,21.277778
Africa,15.413793


In [18]:
#Point system
wght_gld=10
wght_slv=5
wght_brz=2

#pnts=(pts per gold*num of gold) + (pts per silver*num of silver) + (pts per bronze*num of bronze) 
w_continent["Total_Summer_Points"]=wght_gld*w_continent["Summer_Gold"]+wght_slv*w_continent["Summer_Silver"]+wght_brz*w_continent["Summer_Bronze"]
w_continent["Total_Winter_Points"]=wght_gld*w_continent["Winter_Gold"]+wght_slv*w_continent["Winter_Silver"]+wght_brz*w_continent["Winter_Bronze"]
w_continent["Total_Pts"]=w_continent["Total_Summer_Points"]+w_continent["Total_Winter_Points"]

w_continent


,Country (IOC),Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,...,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total,Continent,Country,IOC,Total_Summer_Points,Total_Winter_Points,Total_Pts
0,Afghanistan (AFG),16,0,0,2,2,0,0,0,0,...,0,0,2,2,Asia,Afghanistan,AFG,4,0,4
1,Albania (ALB),10,0,0,2,2,5,0,0,0,...,0,0,2,2,Europe,Albania,ALB,4,0,4
2,Algeria (ALG),15,7,4,9,20,3,0,0,0,...,7,4,9,20,Africa,Algeria,ALG,108,0,108
3,Argentina (ARG),26,22,27,31,80,20,0,0,0,...,22,27,31,80,South America,Argentina,ARG,417,0,417
4,Armenia (ARM),8,2,11,9,22,8,0,0,0,...,2,11,9,22,Asia,Armenia,ARM,93,0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,Zimbabwe (ZIM),15,3,4,1,8,1,0,0,0,...,3,4,1,8,Africa,Zimbabwe,ZIM,52,0,52
158,Individual Neutral Athletes (AIN),1,1,3,1,5,0,0,0,0,...,1,3,1,5,NaN,Individual Neutral Athletes,AIN,27,0,27
159,Independent Olympic Athletes (IOA),3,1,0,1,2,0,0,0,0,...,1,0,1,2,NaN,Independent Olympic Athletes,IOA,12,0,12
160,Independent Olympic Participants (IOP),1,0,1,2,3,0,0,0,0,...,0,1,2,3,NaN,Independent Olympic Participants,IOP,9,0,9


In [19]:
#save modified df  as a csv
#w_continent.to_csv(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Modified.csv", index=False)
w_continent.loc[w_continent["Continent"].notnull()].to_csv(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Modified.csv", index=False)

#save this modified df as an excel file
#w_continent.to_excel(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Modified.xlsx", index=False)
w_continent.loc[w_continent["Continent"].notnull()].to_excel(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Modified.xlsx", index=False)

In [34]:
# GETTING COUNTRIES THAT PARTICIPATED BUT DID NOT GET ANY MEDALS SINCE THEY ARE MISSING FROM THE DATAFRAME(1st table)

no_medal_table=soup.find_all("table")[2]
no_medal_titles=no_medal_table.find_all("th")
cleaned_no_medal_titles=[title.text.strip() for title in no_medal_titles]

#change values in the list to the given values
cleaned_no_medal_titles[1:]=["Summer_Olympics_Attended","Winter_Olympics_Attended","Total_Olympics_Attended"]

no_medal_data_draft=no_medal_table.find_all("tr")[1:]#first row are column names which arent needed


#empty list to hold row data values
no_medal_rows=[]

for row in no_medal_data_draft:
    #find all "td" tags
    row_data=row.find_all("td")
    #clean up every element using list comprehension
    row_data_clean=[data.text.strip() for data in row_data]
    
    #remove any chars after the ")" using split and add back the ")"
    #EX:"Australiia (AUS)[AUS][Z]" gets split into ["Australiia (AUS" , "[AUS][Z]"]. We only need element 
    #at index 0, then add a ")" at end. "Australia (AUS" + ")" = "Australia (AUS)"
    fixed=row_data_clean[0].split(")")[0]+")"
    
    #add clean element into the first index of the list
    row_data_clean[0]=fixed
    
    no_medal_rows.append(row_data_clean)

#create df for countries w/o medals
no_medal_df=pd.DataFrame(no_medal_rows, columns=cleaned_no_medal_titles)

#convert object dtypes in df to best possible dtypes(FOR SOME REASON STILL CHOSE STRING FIRST FOR ALL)
new_no_medal_df=no_medal_df.convert_dtypes()

#change all the columns except the one at index 0[country name] to numeric dtype(int64)
no_medal_int_cols=new_no_medal_df.columns[1:]
new_no_medal_df[no_medal_int_cols]=new_no_medal_df[no_medal_int_cols].apply(pd.to_numeric)



new_no_medal_df.rename(columns={"Team (IOC code)":"Country (IOC)"}, inplace=True)

#ADD CONTINENT COLUMN TO THIS DF

#USING METHOD 2 FROM THE "ADDING CONTINENT COLUMN IN A PRIOR SECTION USING THE 'cont_country' dictionary 
#loop through the dictionary
for continent,country_list in cont_country.items():
    #go through each country in the country list
    for cntry in country_list:
        #check to see any countries in df that match the current country in the country list and set the "continent" as
        #the current key being looped through
        new_no_medal_df.loc[new_no_medal_df["Country (IOC)"].str.contains(cntry), "Continent"]=continent



#Add  corresponding "continent" to countries that were missed and had a NaN under 'continent'
#using previous 'continent' list

na_africa=[]
na_asia=["Palestine", "East Timor", "(COR)", "Malaya", "North Borneo"]
na_europe= ["Saar"]
na_north_america=["British Virgin Islands", "Cayman Islands"]
na_south_america=["Aruba"]
na_oceania=["Guam"]

joined_no_medal_countries=[
    na_africa,
    na_asia,
    na_europe,
    na_north_america,
    na_south_america,
    na_oceania
]

#loop through the list of lists and examine each element and see if any match, and add corresponding "continent"
for i in range(num_groups):
    for no_medal_cntry in joined_no_medal_countries[i]:
        new_no_medal_df.loc[new_no_medal_df["Country (IOC)"].str.contains(no_medal_cntry, regex=False), "Continent"]=continents[i]
    
#split 'Country (IOC)' into 2 extra cols
new_no_medal_df[["Country", "IOC"]]=new_no_medal_df["Country (IOC)"].str.split("(", expand=True)
new_no_medal_df["IOC"]=new_no_medal_df["IOC"].str.replace(")", "")
#remove leading and trailing whitespaces
new_no_medal_df["Country"]=new_no_medal_df["Country"].str.strip()


#joining two dataframes(countries w medals and countries without  ie: w_continent + new_no_medal_df)
combined_dfs=pd.concat([w_continent, new_no_medal_df], ignore_index=True, axis=0)


#Main method: AFTER JOINING THE new_no_medal_df that was missing cols, it will have NaN values there, so replace w 0
#replace NaN under the following columns
nan_zero=['Summer_Gold','Summer_Silver', 'Summer_Bronze', 'Summer_Total', 'Winter_Gold', 'Winter_Silver',
          'Winter_Bronze', 'Winter_Total','Combined_Gold', 'Combined_Silver', 'Combined_Bronze', 'Combined_Total',
         'Total_Summer_Points', 'Total_Winter_Points', "Total_Pts"]

for col in nan_zero:
    combined_dfs.loc[combined_dfs[col].isnull(), col]=0


#METHOD 2 Add the missing columns and set values equal to zero to the new_no_medal_df before concatenating/stack/join
#with the w_continent df so we do not get NaN and then have to switch the NaN values
"""

#replace NaN under the following columns
nan_zero=['Summer_Gold','Summer_Silver', 'Summer_Bronze', 'Summer_Total', 'Winter_Gold', 'Winter_Silver',
          'Winter_Bronze', 'Winter_Total','Combined_Gold', 'Combined_Silver', 'Combined_Bronze', 'Combined_Total',
         'Total_Summer_Points', 'Total_Winter_Points', "Total_Pts"]

for col in nan_zero:
    new_no_medal_df[col]=0


#joining two dataframes(countries w medals and countries without  ie: w_continent + new_no_medal_df)
combined_dfs=pd.concat([w_continent, new_no_medal_df], ignore_index=True, axis=0)
combined_dfs

"""
combined_dfs["Country (IOC)"]=combined_dfs["Country (IOC)"].str.strip()
combined_dfs

,Country (IOC),Summer_Olympics_Attended,Summer_Gold,Summer_Silver,Summer_Bronze,Summer_Total,Winter_Olympics_Attended,Winter_Gold,Winter_Silver,Winter_Bronze,...,Combined_Gold,Combined_Silver,Combined_Bronze,Combined_Total,Continent,Country,IOC,Total_Summer_Points,Total_Winter_Points,Total_Pts
0,Afghanistan (AFG),16,0,0,2,2,0,0,0,0,...,0,0,2,2,Asia,Afghanistan,AFG,4,0,4
1,Albania (ALB),10,0,0,2,2,5,0,0,0,...,0,0,2,2,Europe,Albania,ALB,4,0,4
2,Algeria (ALG),15,7,4,9,20,3,0,0,0,...,7,4,9,20,Africa,Algeria,ALG,108,0,108
3,Argentina (ARG),26,22,27,31,80,20,0,0,0,...,22,27,31,80,South America,Argentina,ARG,417,0,417
4,Armenia (ARM),8,2,11,9,22,8,0,0,0,...,2,11,9,22,Asia,Armenia,ARM,93,0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,North Borneo (NBO),1,0,0,0,0,0,0,0,0,...,0,0,0,0,Asia,North Borneo,NBO,0,0,0
229,Republic of China (ROC),3,0,0,0,0,0,0,0,0,...,0,0,0,0,Asia,Republic of China,ROC,0,0,0
230,Saar (SAA),1,0,0,0,0,0,0,0,0,...,0,0,0,0,Europe,Saar,SAA,0,0,0
231,North Yemen (YAR),2,0,0,0,0,0,0,0,0,...,0,0,0,0,Asia,North Yemen,YAR,0,0,0


In [64]:
#CLEANING UP AN ISSUE UNDER THE HOOD THAT COULD ONLY BE SEEN UPON CLOSER INSPECTION, EXPLANATION BELOW
"""
BEFORE CLEANING UP THE 'Country (IOC)' COLUMN IT WOULD BE LIKE 'CountryName\xa0(IOC)' THROUGHOUT BUT WHEN RETURNING THE
DF IT WOULDNT BE SEEN, IT WOULD LOOK LIKE 'CountryName (IOC)', BUT IT WOULD BE SEEN WHEN DOING AN ILOC RETURN OF THE NAME
FOR INSTANCE. 

HOWEVER, THE '\xa0' DOESNT APPEAR IN THE 'Country' or 'IOC' columns THAT WERE CREATED BY SPLITTING 'Country (IOC)' ON
THE '('

EXAMPLE:
#combined_dfs.iloc[87,0] #returns 'Mexico\xa0(MEX)'

#to get the row with Country (IOC) is 'Mexico (MEX)', we have to use "Mexico\xa0(MEX)" because when it was scraped the 
#'\xa0' was included instead of a space 
#combined_dfs[combined_dfs["Country (IOC)"].str.contains("Mexico\xa0(MEX)", regex=False)]
"""


#cleaning up the and removing '\xa0'(NOTE: FOR SOME REASON IT DOESN'T APPEAR IN THE COUNTRY COLUMN OR THE IOC COLUMN)
combined_dfs["Country (IOC)"]=combined_dfs["Country (IOC)"].str.replace("\xa0", " ")
combined_dfs.iloc[87,:] #Testing to see if it was fixed

#another way to search{searching for Korea (COR)}
#combined_dfs[combined_dfs["Country (IOC)"].isin(["Korea (COR)"])]

Country (IOC)                Mexico (MEX)
Summer_Olympics_Attended               25
Summer_Gold                            13
Summer_Silver                          27
Summer_Bronze                          37
Summer_Total                           77
Winter_Olympics_Attended               10
Winter_Gold                             0
Winter_Silver                           0
Winter_Bronze                           0
Winter_Total                            0
Total_Olympics_Attended                35
Combined_Gold                          13
Combined_Silver                        27
Combined_Bronze                        37
Combined_Total                         77
Continent                   North America
Country                            Mexico
IOC                                   MEX
Total_Summer_Points                   339
Total_Winter_Points                     0
Total_Pts                             339
Name: 87, dtype: object

In [65]:
#EXAMINING COUNTRIES WHOSE NAMES HAVE CHANGED, BROKE UP, GAINED INDEPENDENCE, OR JOINED OTHER COUNTRIES
"""
msng_afr=["Ivory Coast"]
msng_asi=["Hong Kong", "(TPE)"]
msng_eur=["Bohemia","Czech Republic", "Czechoslovakia","Great Britain","Soviet Union", "(ROC)", "(YUG)"]
msng_na=["Bahamas", "Bermuda", "British West Indies","(PUR)", "(ISV)"]
msng_sa=[]
msng_oce=[]

na_africa=[]
na_asia=["Palestine", "East Timor", "(COR)", "Malaya", "North Borneo"]
na_europe= ["Saar"]
na_north_america=["British Virgin Islands", "Cayman Islands"]
na_south_america=["Aruba"]
na_oceania=["Guam"]
"""

old_country_names=["Czechoslovakia", "Soviet Union", "Yugoslavia", "British West Indies", "(COR)", "Malaya",
                   "North Borneo", "Saar"]


In [66]:
#save combined df[only countries ..special delegations have continent as NaN & not included]  as a csv
#combined_dfs.to_csv(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Modified.csv", index=False)
combined_dfs.loc[combined_dfs["Continent"].notnull()].to_csv(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_All_Countries.csv", index=False)

#save this modified df as an excel file
#combined_dfs.to_excel(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_Modified.xlsx", index=False)
combined_dfs.loc[combined_dfs["Continent"].notnull()].to_excel(r"C:\Users\wisec\Desktop\Total_Medals_Olympics_All_Countries.xlsx", index=False)